
## Exercise 4: Fisher Score

**Problem Description:**  
Measure class separability: ratio of between-class to within-class variance.

**Solution Overview:**  
Implement Fisher score formula and rank features.



In [ ]:
!pip install scikit-learn pandas matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.feature_selection import VarianceThreshold


In [ ]:
# 4.1 Compute Fisher scores
def fisher_score(X_df, y_arr):
    scores = []
    classes = np.unique(y_arr)
    for col in X_df.columns:
        overall_mean = X_df[col].mean()
        num = den = 0
        for cls in classes:
            Xi = X_df[col][y_arr == cls]
            num += len(Xi) * (Xi.mean() - overall_mean) ** 2
            den += len(Xi) * Xi.var()
        scores.append(num / den if den else 0)
    return scores

fs = fisher_score(X, y)
fs_df = pd.DataFrame({'feature': X.columns, 'Fisher_score': fs}).sort_values(by='Fisher_score', ascending=False)
print(fs_df)


# 4.2 Analysis
- Derive the Fisher score formula from between-class and within-class variances.

- Compare Fisher scores to ANOVA F-scores.

- Visualize Fisher scores with a bar plot.

# Exercise 4: Fisher Score
## Analysis to Include in Code

In [ ]:
# Formula Verification
# Recompute between/within variances separately for one feature
col = X.columns[0]
# … compute num, den as in fisher_score(), then compare:
print("Manual vs. function:", manual_score, fs[0])


In [ ]:
# Correlation with ANOVA

pd.DataFrame({'ANOVA':anova_df['F_score'], 'Fisher':df_fs['Fisher_score']}).corr()


In [ ]:
# Class-wise Distributions

top = df_fs['feature'].iloc[0]
sns.boxplot(x=y, y=X[top]); plt.title(f"Class separation on {top}"); plt.show()
